# Arctic Amplification and New York State Temperature Anomalies

Temperature Anomalies using CFSR Data

---

## Overview

Introductory paragraph here ... blah blah AA using CFSR Data, and NYS using NYSM w only 6 years..

1. Section 1: Create Monthly Mean Datasets by Year from 1979-2022 using CFSR Data
2. Section 2: Create Temperature Anomaly & Arctic Amplification Figures using CFSR Data
3. Third Content Section

## Prerequisites
Not sure if I want to add these

---

## Imports

In [1]:
import xarray as xr
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import datetime #remove for final version
import os


## Section 1: Create Monthly Mean Datasets by Year from 1979-2022 using CFSR Data

This section is for ***Reference Only*** because the monthly mean computations take a long time to run at ~6 minutes per year (over 4 hours for the full 43 years of data). This code has already been ran and net cdf files have been saved out locally, so those files can be read in and used as pre-computed datasets for later parts of this notebook.

Data source: Climate Forecast System Renanalysis, https://climatedataguide.ucar.edu/climate-data/climate-forecast-system-reanalysis-cfsr

Local copies (/cfsr/data/) of CFSR datasets were used in this notebook.

Referenced notebooks: UAlbany ATM622 Jupyter notebook https://brian-rose.github.io/general-circulation/lectures/computing-seasonal.html
 

### Define functions
Functions are used when creating and saving each monthly mean dataset by year

In [ ]:
def make_dir(path):
    """ 
    Input directory path as string
    Creates the directory if it doesn't already exist
    """ 
    if not os.path.exists(path):
        os.makedirs(path)
        
def open_ds(yr):
    """ Open dataset using dask """
    ds = xr.open_mfdataset(f'/cfsr/data/{yr}/{var}.{yr}.0p5.anl.nc', chunks={'time':30*4, 'lev': 4}, parallel=True) # removed this before /cfsr "/network/daes"
    return ds

def compute_save_means(ds_for_mean, yr):
    
    """ 
    Input dataset based depends on whether grouping seasonally, annually, etc.
    Perform lazy execution averaging on the input dataset
    Calculation is executed when saving to path
    """
    ds_mean = ds_for_mean.mean(dim=('lon','time'), skipna=True)
    save_path = f'{save_dir}/{group_desc}_{var}_{yr}.nc'
    ds_mean.to_netcdf(save_path)
    print(save_path) #comment out for final version
    print(f"finished {yr} at {datetime.datetime.now()}") #comment out for final version
    

### Create and save monthly mean temperature datasets for each year from 1979-2022

#### Define Variables

In [ ]:
# Define CFSR variable of interest (e.g. temperature is 't') 
var = 't' 

# Describes how data should be grouped for averaging, used in file and directory names
group_desc = 'monthly' 

# Directory where averaged net cdf files will be saved out
save_dir = f'/home11/grad/2021/cs436778/general-circulation/project/data/{group_desc}'

# Years of CFSR data to include; each will be looped over
years = range(1979, 2023)

#### Execute monthly mean calculation for each year and save dataset

In [ ]:
# execute function monthl

make_dir(save_dir)

for year in years:

    ds = open_ds(year)

    # group dataframe depending on seasonal, annual, monthly means
    ds_grouped = ds.groupby(ds.time.dt.month)

    compute_save_means(ds_grouped, year)

## Section 2: Create Temperature Anomaly & Arctic Amplification Figures using CFSR Data

Recreate figures 1A and 2A from **Francis & Vavrus 2015**. Citation and link to paper: Jennifer A Francis and Stephen J Vavrus 2015 Environ. Res. Lett. 10 014005,  https://iopscience.iop.org/article/10.1088/1748-9326/10/1/014005#erl507077bib8

### Read in pre-computed monthly mean data

In [2]:
# Define the parent dir where annual temp data lives
# dir_seasonal_t = '/home11/grad/2021/cs436778/general-circulation/project/data/seasonal/'
dir_annual_t = '/home11/grad/2021/cs436778/general-circulation/project/data/annual/'
dir_monthly_t = '/home11/grad/2021/cs436778/general-circulation/project/data/monthly/'

# Define the years of data to read in. As discussed above, we will read in 2002 through 2021
years = range(1979,2022)

leap = ['1980', '1984', '1988', '1992', '1996', '2000', '2004', '2008', '2012', '2016', '2020']

### Calculate Annual and Seasonal long-term averages

#### Define Functions 

In [3]:
def latrange_mean(dataset, elev_p, latrange):
    """ 
    For each year, read in the seasonal dataset and subset by elevation and latitude range (ex: arctic and mid latitudes)
    Calculate the latitude-weighted mean temperature for each season
    """
    latrange = dataset.t.sel(lev=elev_p, lat = latrange)
    latrange_weights = np.cos(np.deg2rad(latrange.lat))
    latrange_mean = latrange.weighted(latrange_weights).mean(dim = 'lat')
    return latrange_mean

def add_to_dict(dict_name, key_name, val):
    """ Append a value to a key in a dictionary if the key already exists
    Otherwise, if key doesn’t exist in dict yet, add a new key and its value to the dictionary 
    """
    if key_name in dict_name:
        if type(dict_name[key_name]) is not list:
            key_current_val = [dict_name[key_name]]
        else:
            key_current_val = dict_name[key_name]
        key_current_val.append(val)
        dict_name[key_name] = key_current_val
    else:
        dict_name[key_name] = val    
        
def yearly_monthly_means(dataset, dict_name):
    """
    For each season, track the year, season, and mean value (three keys)
    Returns a dictionary of the tracked data 
    """
    for month_num in range(1,13):
        month_val = dataset.sel(month = month_num).values.item()
        add_to_dict(dict_name, month_num, month_val)
        
def year_month_length(yr, dictname):
    """
    Identifies the length of day in a month or year depending on leap year
    Note: These lengths will be used for calculating seasonal and annual climatologies rather than using xarray's built in averaging over 'time.season' which is DJF rather than JFM
    """
    if yr in leap:
        add_to_dict(dictname, 'count_2', 29)
        add_to_dict(dictname, 'count_year', 366)
    else: 
        add_to_dict(dictname, 'count_2', 28)
        add_to_dict(dictname, 'count_year', 365)
    add_to_dict(dictname, 'count_1', 31)
    add_to_dict(dictname, 'count_3', 31)
    add_to_dict(dictname, 'count_4', 30)
    add_to_dict(dictname, 'count_5', 31)
    add_to_dict(dictname, 'count_6', 30)
    add_to_dict(dictname, 'count_7', 31)
    add_to_dict(dictname, 'count_8', 31)
    add_to_dict(dictname, 'count_9', 30)
    add_to_dict(dictname, 'count_10', 31)
    add_to_dict(dictname, 'count_11', 30)
    add_to_dict(dictname, 'count_12', 31)

# Create dictionaries of data by month and year for a given latitude range and pressure level
def dict_month_data(plevel, lat_low, lat_high):
    new_dict = {}
    for year in range(1979,2022):
        
        # add year to dictionary
        year = str(year)
        add_to_dict(new_dict, 'year_name', year)
        
        # open datasets
        ds_year_month = xr.open_mfdataset(f"{dir_monthly_t}monthly_t_{year}.nc")
        ds_year = xr.open_mfdataset(f"{dir_annual_t}annual_t_{year}.nc")

        # subset datasets
        lat_avg_months = latrange_mean(ds_year_month, plevel, slice(lat_low,lat_high))
        lat_avg_year = latrange_mean(ds_year, plevel, slice(lat_low,lat_high))
        
        # add subsetted data to dictionary
        yearly_monthly_means(lat_avg_months, new_dict)
        add_to_dict(new_dict, 'annual', lat_avg_year.values.item())
        
        # add month length and year length to dictionary accounting for leap years
        year_month_length(year, new_dict)
            
    return new_dict

# calculate seasonaly and annual climatologies, used for calculating anomalous temp
def seasonal_climatology(region_dict, months_ls):
    """ 
    Input regional dictionary of temps by month (e.g. arctic_dict made above) 
    Input list of month numbers that should be averaged to create season (e.g. input list [1,2,3] to average Jan, Feb, Mar)
    Returns average value for season 
    """
    month_values = region_dict[months_ls[0]] + region_dict[months_ls[1]] + region_dict[months_ls[2]]
    month_weights = region_dict[f"count_{str(months_ls[0])}"] + region_dict[f"count_{str(months_ls[1])}"] + region_dict[f"count_{str(months_ls[2])}"]
    season_mean = np.average(month_values, weights = month_weights)
    return season_mean

#### Create dictionaries 

In [4]:
midlat_dict_new = dict_month_data(1000, 30, 60)
arctic_dict_new = dict_month_data(1000, 70, 90)

In [ ]:
arctic_dict_new

In [5]:
arctic_dict_new.keys()

dict_keys(['year_name', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 'annual', 'count_2', 'count_year', 'count_1', 'count_3', 'count_4', 'count_5', 'count_6', 'count_7', 'count_8', 'count_9', 'count_10', 'count_11', 'count_12'])

In [22]:
# # create dictionaries which averages will be appended to
# midlat_dict = {}
# arctic_dict = {}

# # Add seasonal data for each year from 1979-2021 
# for year in range(1979,2022):
#     """ 
#     Open dataset for each year
#     """
#     year = str(year)
    
#     # add year
#     add_to_dict(midlat_dict, 'year_name', year)
#     add_to_dict(arctic_dict, 'year_name', year)
                  
#     # open dataset
#     ds_year = xr.open_mfdataset(f"{dir_annual_t}annual_t_{year}.nc")
#     ds_year_month = xr.open_mfdataset(f"{dir_monthly_t}monthly_t_{year}.nc")

#     # middle latitudes - annual
#     midlat_mean_year = latrange_mean(ds_year, 1000, slice(30,60))
#     add_to_dict(midlat_dict, 'annual', midlat_mean_year.values.item())
#     # middle latitudes - seasonal
#     midlat_mean_month = latrange_mean(ds_year_month, 1000, slice(30,60))
#     yearly_monthly_means(midlat_mean_month, midlat_dict)

#     # arctic - annual
#     arctic_mean_year = latrange_mean(ds_year, 1000, slice(70,90))
#     add_to_dict(arctic_dict, 'annual', arctic_mean_year.values.item())
#     # arctic - seasonal
#     arctic_mean_month = latrange_mean(ds_year_month, 1000, slice(70,90))
#     yearly_monthly_means(arctic_mean_month, arctic_dict)
    
#     # add fields for ct_feb and ct_yr depending if leap year. Used in averaging
#     year_month_length(year)



In [6]:


arctic_jfm_mean = seasonal_climatology(arctic_dict_new, [1,2,3])
midlat_jfm_mean = seasonal_climatology(midlat_dict_new, [1,2,3])

arctic_amj_mean = seasonal_climatology(arctic_dict_new, [4,5,6])
midlat_amj_mean = seasonal_climatology(midlat_dict_new, [4,5,6])

arctic_jas_mean = seasonal_climatology(arctic_dict_new, [7,8,9])
midlat_jas_mean = seasonal_climatology(midlat_dict_new, [7,8,9])

arctic_ond_mean = seasonal_climatology(arctic_dict_new, [10,11,12])
midlat_ond_mean = seasonal_climatology(midlat_dict_new, [10,11,12])


In [45]:
# seasonally
arctic_jfm_tomean = arctic_dict_new[1] + arctic_dict_new[2] + arctic_dict_new[3]
arctic_jfm_weights = arctic_dict_new['count_1'] + arctic_dict_new['count_2'] + arctic_dict_new['count_3']
arctic_jfm= np.average(arctic_jfm_tomean, weights = arctic_jfm_weights)


In [8]:
arctic_jfm_mean

252.0783331934185

In [27]:
# calculate averages from monthly data (average across years) - need this for relative for anom calc

# seasonally
arctic_jfm_tomean = arctic_dict[1] + arctic_dict[2] + arctic_dict[3]
arctic_jfm_weights = arctic_dict['c1'] + arctic_dict['c2'] + arctic_dict['c3']
arctic_jfm= np.average(arctic_jfm_tomean, weights = arctic_jfm_weights)

# arctic_amj_tomean = arctic_dict[4] + arctic_dict[5] + arctic_dict[6]
# arctic_amj_weights = arctic_dict['c4'] + arctic_dict['c5'] + arctic_dict['c6']
# arctic_amj= np.average(arctic_amj_tomean, weights = arctic_amj_weights)

# arctic_jas_tomean = arctic_dict[7] + arctic_dict[8] + arctic_dict[9]
# arctic_jas_weights = arctic_dict['c7'] + arctic_dict['c8'] + arctic_dict['c9']
# arctic_jas= np.average(arctic_jas_tomean, weights = arctic_jas_weights)

# arctic_ond_tomean = arctic_dict[10] + arctic_dict[11] + arctic_dict[12]
# arctic_ond_weights = arctic_dict['c10'] + arctic_dict['c11'] + arctic_dict['c12']
# arctic_ond= np.average(arctic_ond_tomean, weights = arctic_ond_weights)

# midlat_jfm_tomean = midlat_dict[1] + midlat_dict[2] + midlat_dict[3]
# midlat_jfm_weights = midlat_dict['c1'] + midlat_dict['c2'] + midlat_dict['c3']
# midlat_jfm= np.average(midlat_jfm_tomean, weights = midlat_jfm_weights)

# midlat_amj_tomean = midlat_dict[4] + midlat_dict[5] + midlat_dict[6]
# midlat_amj_weights = midlat_dict['c4'] + midlat_dict['c5'] + midlat_dict['c6']
# midlat_amj= np.average(midlat_amj_tomean, weights = midlat_amj_weights)

# midlat_jas_tomean = midlat_dict[7] + midlat_dict[8] + midlat_dict[9]
# midlat_jas_weights = midlat_dict['c7'] + midlat_dict['c8'] + midlat_dict['c9']
# midlat_jas= np.average(midlat_jas_tomean, weights = midlat_jas_weights)

# midlat_ond_tomean = midlat_dict[10] + midlat_dict[11] + midlat_dict[12]
# midlat_ond_weights = midlat_dict['c10'] + midlat_dict['c11'] + midlat_dict['c12']
# midlat_ond= np.average(midlat_ond_tomean, weights = midlat_ond_weights)

# # yearly
# arctic_ann_tomean = arctic_jfm_tomean+arctic_amj_tomean+arctic_jas_tomean+arctic_ond_tomean
# arctic_ann_weights = arctic_jfm_weights+arctic_amj_weights+arctic_jas_weights+arctic_ond_weights
# arctic_ann= np.average(arctic_ann_tomean, weights = arctic_ann_weights)

# midlat_ann_tomean = midlat_jfm_tomean+midlat_amj_tomean+midlat_jas_tomean+midlat_ond_tomean
# midlat_ann_weights = midlat_jfm_weights+midlat_amj_weights+midlat_jas_weights+midlat_ond_weights
# midlat_ann= np.average(midlat_ann_tomean, weights = midlat_ann_weights)


TypeError: Axis must be specified when shapes of a and weights differ.

### Plot annual-mean anomalies in air temperature (Francis & Vavrus 2015) Plot 1A

Plot 1: Annual-mean anomalies in air temperature for 40–80°N (recreating figure 1a from Francis & Vavrus)
- "Annual anomaly" is defined as the difference in annual mean relative to the **1989-2018 mean** (30 years)
    - Note that this is updated from Francis & Vavrus' available data from 1981–2010 (30 years)
- Anomalies in air temperature are calculated for each year from **2003 through 2021** (19 years) and averaged to get annual-mean anomalies which are plotted by latitude and elevation. 
    - Note that this is updated from Francis & Vavrus' anomalies calculatued for 1995 through 2013 (19 years).
- Thus, all data ranges have been shifted forward 8 years

### Plot 2A

 Arctic amplification seasonal time series
- Arctic Amplification defined as the difference in 1000 hPa temperature anomalies (relative to 1979–2021 mean), between the Arctic (70–90°N) and mid-latitudes (30–60°N)
    - Note that this is updated from Francis & Vavrus' available data from 1948–2013 mean

- Q: need seasonal climatologies or just the seasonal means by year?

## Third Content Section

___

## Summary

Paragraph